# AWS Lambda から CloudWatch Observability を使用して Amazon Bedrock AgentCore Runtime を呼び出す

## 概要

このチュートリアルでは、CloudWatch 可観測性を有効にした状態で、AWS Lambda 関数から Amazon Bedrock AgentCore Runtime でホストされている Model Context Protocol (MCP) サーバーを持つ Strands エージェントを呼び出す方法を説明します。

### チュートリアル詳細

| 項目         | 詳細                                                                          |
|:-------------------|:----------------------------------------------------------------------------------|
| チュートリアルタイプ      | 会話型                                                                   |
| エージェントタイプ         | シングル                                                                           |
| エージェントフレームワーク  | Strands Agents                                                                   |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                      |
| チュートリアルコンポーネント| Lambda 呼び出し、AgentCore Runtime、MCP サーバー、CloudWatch Observability     |
| 難易度 | 上級                                                                         |
| 使用 SDK           | Amazon BedrockAgentCore Python SDK、boto3、AWS Lambda                           |

### アーキテクチャ
```
┌─────────┐      ┌────────────────┐      ┌──────────────────┐      ┌─────────────────┐
│   API   │─────>│  AWS Lambda    │─────>│  AgentCore       │─────>│  Strands Agent  │
│  /User  │      │  (Invoker)     │      │  Runtime         │      │  + MCP Servers  │
└─────────┘      └────────────────┘      └──────────────────┘      └─────────────────┘
                        │                         │                          │
                        ▼                         ▼                          ▼
                 ┌────────────────────────────────────────────────────────────┐
                 │            CloudWatch Observability                        │
                 │       • Gen AI トレース     • メトリクス     • ログ             │
                 └────────────────────────────────────────────────────────────┘
```

### 主な機能

* 複数の MCP サーバー（AWS Documentation + AWS CDK）と Strands Agents の統合
* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* AWS Lambda 関数からホストされたエージェントの呼び出し
* CloudWatch Gen AI Observability によるエージェント実行の監視
* AWS Lambda Layer for OpenTelemetry によるエンドツーエンドのトレース伝播

### 学習内容

1. MCP 対応エージェントを AgentCore Runtime にデプロイする方法
2. Runtime エージェントを呼び出す Lambda 関数を作成する方法
3. エージェントの CloudWatch Gen AI Observability を有効にする方法
4. トレース伝播のために ADOT Lambda Layer を設定する方法
5. CloudWatch コンソールでトレースとログを表示する方法

## 前提条件

### 必要なソフトウェア
* Python 3.10+
* 適切な権限を持つ AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Lambda 関数と IAM ロールを作成する権限

### CloudWatch Transaction Search の有効化（一回限りのセットアップ）

**このチュートリアルを開始する前に、CloudWatch で Transaction Search を有効にする必要があります。** これは AWS アカウントとリージョンごとに一回限りのセットアップです。

#### Transaction Search を有効にする手順：

1. **CloudWatch コンソールに移動**
   - https://console.aws.amazon.com/cloudwatch/ にアクセス
   - リージョンを選択（右上隅）

2. **Gen AI Observability を開く**
   - 左側のナビゲーションで、**Application Signals** までスクロール
   - **Gen AI Observability** をクリック

3. **Transaction Search を有効化**
   - まだ有効になっていない場合、Transaction Search を有効にするバナーが表示されます
   - **Enable Transaction Search** をクリック
   - アクションを確認

4. **セットアップが完了するまで待機**
   - Transaction Search が完全に動作するまで約 **10 分** かかります
   - セットアップ中にチュートリアルを続行できます

**重要な注意事項：**
- Transaction Search は Gen AI Observability 機能に必要です
- 無料枠にはスパンの 1% サンプリングが含まれます
- 有効化すると、アカウント/リージョン内のすべての AI ワークロードに適用されます

### 必要なパッケージのインストール

In [ ]:
!pip install --upgrade "strands-agents[otel]" strands-agents-tools boto3 bedrock-agentcore bedrock-agentcore-starter-toolkit uv

## ステップ 1: AWS セッションの初期化とアカウント情報の取得

In [ ]:
import boto3
import json
import time
from boto3.session import Session

# Initialize session and get account details
boto_session = Session()
region = boto_session.region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']

print(f"AWS Account ID: {account_id}")
print(f"Region: {region}")

## ステップ 2: 複数のサーバーを使用する MCP エージェントの作成

2 つの MCP サーバーを使用するエージェントを作成します：
1. AWS Documentation MCP Server - AWS ドキュメントへのアクセス用
2. AWS CDK MCP Server - CDK のベストプラクティスとガイダンス用

In [ ]:
%%writefile mcp_agent_multi_server.py
from strands import Agent
from strands.models import BedrockModel
from mcp import StdioServerParameters, stdio_client
from strands.tools.mcp import MCPClient
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# BedrockAgentCoreApp を初期化
app = BedrockAgentCoreApp()

# AWS Documentation MCP サーバーに接続
def create_aws_docs_client():
    return MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx", 
                args=["awslabs.aws-documentation-mcp-server@latest"]
            )
        )
    )

# AWS CDK MCP サーバーに接続
def create_cdk_client():
    return MCPClient(
        lambda: stdio_client(
            StdioServerParameters(
                command="uvx", 
                args=["awslabs.cdk-mcp-server@latest"]
            )
        )
    )

# 両方の MCP サーバーからツールを持つエージェントを作成する関数
def create_agent():
    model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
    model = BedrockModel(model_id=model_id)
    
    aws_docs_client = create_aws_docs_client()
    cdk_client = create_cdk_client()
    
    with aws_docs_client, cdk_client:
        # 両方の MCP サーバーからツールを取得
        tools = aws_docs_client.list_tools_sync() + cdk_client.list_tools_sync()
        
        # これらのツールを持つエージェントを作成
        agent = Agent(
            model=model,
            tools=tools,
            system_prompt="""あなたは AWS ドキュメントと CDK のベストプラクティスにアクセスできる、親切な AWS アシスタントです。
            AWS サービスとインフラストラクチャ as コードパターンに関する簡潔で正確な情報を提供してください。
            料金や CDK について質問された場合は、ツールを使用して最新の情報を検索してください。"""
        )
    
    return agent, aws_docs_client, cdk_client

@app.entrypoint
def invoke_agent(payload):
    """入力ペイロードを処理し、エージェントのレスポンスを返します"""
    agent, aws_docs_client, cdk_client = create_agent()
    
    with aws_docs_client, cdk_client:
        user_input = payload.get("prompt")
        print(f"リクエスト処理中: {user_input}")
        response = agent(user_input)
        return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

In [ ]:
%%writefile requirements.txt
strands-agents[otel]
strands-agents-tools
uv
boto3
bedrock-agentcore
aws-opentelemetry-distro==0.12.1

## ステップ 3: エージェントを AgentCore Runtime にデプロイ

MCP エージェントを AgentCore Runtime にデプロイします。Runtime は CloudWatch 可観測性のために OpenTelemetry でエージェントを自動的にインストゥルメントします。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

# Runtime を設定
agentcore_runtime = Runtime()
agent_name = "mcp_agent_lambda_observability"

# 既存のエージェントがあるか確認
bedrock_agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

try:
    # 既存のエージェントをリストして確認
    print("Checking for existing agent...")
    
    # configure は既存のエージェントを適切に処理します
    config_response = agentcore_runtime.configure(
        entrypoint="mcp_agent_multi_server.py",
        auto_create_ecr=True,
        auto_create_execution_role=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name=agent_name
    )
    
    print(f"Agent configured: {agent_name}")
    
except Exception as e:
    print(f"Configuration note: {e}")
    print("Continuing with existing configuration...")

In [ ]:
# エージェントを AgentCore Runtime に起動または更新
print("Deploying/Updating agent to AgentCore Runtime (this may take several minutes)...")

try:
    # 冪等なデプロイのために auto_update_on_conflict=True を使用
    launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
    
    print(f"\nAgent deployed successfully!")
    print(f"Agent ARN: {launch_result.agent_arn}")
    print(f"Agent ID: {launch_result.agent_id}")
    
    # エージェント詳細を保存
    agent_arn = launch_result.agent_arn
    agent_id = launch_result.agent_id
    
except Exception as e:
    # フォールバック：launch が失敗した場合、既存のエージェントのステータスを取得
    if 'already exists' in str(e).lower() or 'conflict' in str(e).lower():
        print("Agent already exists, retrieving existing agent details...")
        try:
            status_response = agentcore_runtime.status()
            
            # エージェント情報を安全に抽出
            if status_response and hasattr(status_response, 'endpoint') and status_response.endpoint:
                agent_arn = status_response.endpoint.get('agentArn', '')
                if not agent_arn:
                    # 代替フィールド名を試行
                    agent_arn = status_response.endpoint.get('agentRuntimeArn', '')
                agent_id = agent_name
                print(f"Using existing agent: {agent_arn}")
            else:
                # ステータスにエンドポイント情報がない場合、agent_name を使用
                print("Could not retrieve full agent details, using agent name")
                agent_id = agent_name
                agent_arn = ""  # 次のステップで設定される
                
        except Exception as status_error:
            print(f"Status retrieval error: {status_error}")
            print("Using agent name as fallback")
            agent_id = agent_name
            agent_arn = ""
    else:
        print(f"Deployment error: {e}")
        raise e

In [ ]:
# エージェントが準備完了になるまで待機
print("Waiting for agent endpoint to be ready...")
status_response = agentcore_runtime.status()
status = status_response.endpoint.get('status', 'UNKNOWN')
end_statuses = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']

max_attempts = 60
attempt = 0

while status not in end_statuses and attempt < max_attempts:
    time.sleep(10)
    try:
        status_response = agentcore_runtime.status()
        status = status_response.endpoint.get('status', 'UNKNOWN')
        print(f"Status: {status}")
    except Exception as e:
        print(f"Status check error: {e}")
        break
    attempt += 1

if status == 'READY':
    print("\nAgent is ready to accept invocations!")
else:
    print(f"\nAgent status: {status}")

# Runtime ARN を取得（これが Lambda に必要）
agent_runtime_arn = status_response.endpoint.get('agentRuntimeArn', '')
if not agent_runtime_arn:
    agent_runtime_arn = status_response.endpoint.get('endpointArn', '')

print(f"Runtime ARN: {agent_runtime_arn}")

## ステップ 4: 直接呼び出しのテスト

まずエージェントを直接テストして、正しく動作していることを確認しましょう。

In [ ]:
# 直接呼び出しのテスト
test_payload = {"prompt": "What is Amazon Bedrock's pricing model?"}

print(f"Testing agent with prompt: {test_payload['prompt']}\n")

try:
    invoke_response = agentcore_runtime.invoke(test_payload)
    
    # レスポンスを表示
    from IPython.display import Markdown, display
    response_text = invoke_response['response'][0]
    display(Markdown(response_text))
except Exception as e:
    print(f"Direct invocation test error: {e}")
    print("This may be normal if the agent is still initializing. Continue to next step.")

## ステップ 5: Lambda 実行ロールの作成

Lambda 関数には以下の権限が必要です：
1. AgentCore Runtime エージェントの呼び出し
2. CloudWatch へのログ書き込み
3. X-Ray へのトレース送信
4. 拡張可観測性のための Application Signals の使用

In [ ]:
iam_client = boto3.client('iam')

# Lambda 実行ロール名を定義
lambda_role_name = f"AgentCoreLambdaExecutionRole-{agent_name}"

# Lambda の信頼ポリシー
lambda_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# ロールを作成または取得
try:
    role_response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
        Description='Execution role for Lambda to invoke AgentCore Runtime with observability'
    )
    lambda_role_arn = role_response['Role']['Arn']
    print(f"Created new Lambda execution role: {lambda_role_arn}")
    time.sleep(10)  # ロールが伝播するまで待機
except iam_client.exceptions.EntityAlreadyExistsException:
    role_response = iam_client.get_role(RoleName=lambda_role_name)
    lambda_role_arn = role_response['Role']['Arn']
    print(f"Using existing Lambda execution role: {lambda_role_arn}")

# AWS マネージドポリシーをアタッチ
managed_policies = [
    'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole',  # CloudWatch Logs
    'arn:aws:iam::aws:policy/AWSXRayDaemonWriteAccess',  # X-Ray トレーシング
    'arn:aws:iam::aws:policy/CloudWatchLambdaApplicationSignalsExecutionRolePolicy',  # Application Signals
]

for policy_arn in managed_policies:
    try:
        iam_client.attach_role_policy(
            RoleName=lambda_role_name,
            PolicyArn=policy_arn
        )
        print(f"Attached policy: {policy_arn.split('/')[-1]}")
    except iam_client.exceptions.InvalidInputException:
        print(f"Policy already attached: {policy_arn.split('/')[-1]}")
    except Exception as e:
        if 'already attached' in str(e).lower():
            print(f"Policy already attached: {policy_arn.split('/')[-1]}")
        else:
            print(f"Policy attachment note: {e}")

In [ ]:
# AgentCore Runtime 呼び出し用のカスタムポリシーを作成
agentcore_policy_name = f"AgentCoreRuntimeInvokePolicy-{agent_name}"

# スコープダウンポリシー - この特定のエージェントのみ
agentcore_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AgentCoreRuntimeAccess",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:InvokeAgentRuntime"
            ],
            "Resource": agent_runtime_arn  # この特定の Runtime のみ
        },
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": f"arn:aws:bedrock:{region}::foundation-model/*"  # Bedrock モデル
        }
    ]
}

try:
    policy_response = iam_client.create_policy(
        PolicyName=agentcore_policy_name,
        PolicyDocument=json.dumps(agentcore_policy_document),
        Description='Scoped policy to allow Lambda to invoke specific AgentCore Runtime'
    )
    agentcore_policy_arn = policy_response['Policy']['Arn']
    print(f"Created AgentCore invocation policy: {agentcore_policy_arn}")
except iam_client.exceptions.EntityAlreadyExistsException:
    agentcore_policy_arn = f"arn:aws:iam::{account_id}:policy/{agentcore_policy_name}"
    print(f"Using existing AgentCore invocation policy: {agentcore_policy_arn}")
    
    # 既存のポリシードキュメントを更新
    try:
        # 既存のポリシーバージョンを取得
        versions = iam_client.list_policy_versions(PolicyArn=agentcore_policy_arn)['Versions']
        
        # 5 バージョンある場合、最も古い非デフォルトバージョンを削除
        if len(versions) >= 5:
            oldest_version = sorted([v for v in versions if not v['IsDefaultVersion']], 
                                   key=lambda x: x['CreateDate'])[0]
            iam_client.delete_policy_version(
                PolicyArn=agentcore_policy_arn,
                VersionId=oldest_version['VersionId']
            )
        
        # 新しいバージョンを作成
        iam_client.create_policy_version(
            PolicyArn=agentcore_policy_arn,
            PolicyDocument=json.dumps(agentcore_policy_document),
            SetAsDefault=True
        )
        print(f"  Updated policy to latest runtime ARN")
    except Exception as e:
        print(f"  Policy update note: {e}")

# カスタムポリシーをロールにアタッチ
try:
    iam_client.attach_role_policy(
        RoleName=lambda_role_name,
        PolicyArn=agentcore_policy_arn
    )
    print(f"Attached AgentCore invocation policy to Lambda role")
except Exception as e:
    if 'already attached' in str(e).lower():
        print(f"Policy already attached")
    else:
        print(f"Policy attachment note: {e}")

print(f"\nLambda role configured with minimal required permissions")

## ステップ 6: OpenTelemetry 用 AWS Lambda Layer の設定

Lambda から AgentCore Runtime への完全なトレース伝播を有効にするには、AWS Lambda Layer for OpenTelemetry (ADOT) を追加する必要があります。この Layer は：

- Lambda 関数を OpenTelemetry で自動的にインストゥルメント
- ダウンストリームサービス（AgentCore Runtime）にトレースコンテキストを伝播
- CloudWatch トレースでエンドツーエンドの可視性を実現

**この Layer がないと、Lambda と AgentCore Runtime 間のトレースが分断されます。**

In [ ]:
# リージョン別の ADOT Lambda Layer ARN（Python）
# 最新の ARN については https://aws-otel.github.io/docs/getting-started/lambda/lambda-python を参照
adot_layer_arns = {
    'us-east-1': 'arn:aws:lambda:us-east-1:615299751070:layer:AWSOpenTelemetryDistroPython:18',
    'us-east-2': 'arn:aws:lambda:us-east-2:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'us-west-1': 'arn:aws:lambda:us-west-1:615299751070:layer:AWSOpenTelemetryDistroPython:22',
    'us-west-2': 'arn:aws:lambda:us-west-2:615299751070:layer:AWSOpenTelemetryDistroPython:22',
    'ap-south-1': 'arn:aws:lambda:ap-south-1:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'ap-northeast-2': 'arn:aws:lambda:ap-northeast-2:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'ap-southeast-1': 'arn:aws:lambda:ap-southeast-1:615299751070:layer:AWSOpenTelemetryDistroPython:14',
    'ap-southeast-2': 'arn:aws:lambda:ap-southeast-2:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'ap-northeast-1': 'arn:aws:lambda:ap-northeast-1:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'eu-central-1': 'arn:aws:lambda:eu-central-1:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'eu-west-1': 'arn:aws:lambda:eu-west-1:615299751070:layer:AWSOpenTelemetryDistroPython:15',
    'eu-west-2': 'arn:aws:lambda:eu-west-2:615299751070:layer:AWSOpenTelemetryDistroPython:15',
}

adot_layer_arn = adot_layer_arns.get(region)
if not adot_layer_arn:
    print(f"Warning: ADOT Layer ARN not defined for region {region}")
    print(f"Please check https://aws-otel.github.io/docs/getting-started/lambda/lambda-python for the latest ARN")
    print(f"Continuing without ADOT Layer - trace propagation may be limited")
else:
    print(f"Using ADOT Layer for region {region}:")
    print(f"   {adot_layer_arn}")

## ステップ 7: Lambda 関数の作成

この Lambda 関数は、X-Ray トレーシングと ADOT インストゥルメンテーションを有効にして AgentCore Runtime エージェントを呼び出します。

In [ ]:
%%writefile lambda_agentcore_invoker.py
import json
import boto3
import os
import traceback
from botocore.exceptions import ClientError

def lambda_handler(event, context):
    """
    AgentCore Runtime エージェントを呼び出す Lambda 関数。
    
    期待されるイベント形式：
    {
        "prompt": "Your question here",
        "sessionId": "optional-session-id"
    }
    """
    
    # boto3 クライアントを初期化
    bedrock_agentcore_client = boto3.client('bedrock-agentcore')
    
    try:
        # 環境変数を取得
        runtime_arn = os.environ.get('RUNTIME_ARN')
        
        print(f"Lambda function started")
        print(f"Runtime ARN: {runtime_arn}")
        
        if not runtime_arn:
            return {
                'statusCode': 500,
                'body': json.dumps({
                    'error': 'Configuration Error',
                    'message': 'Missing RUNTIME_ARN environment variable'
                })
            }
        
        # 入力を解析
        if isinstance(event, str):
            event = json.loads(event)
        
        prompt = event.get('prompt', '')
        session_id = event.get('sessionId', context.aws_request_id)
        
        if not prompt:
            return {
                'statusCode': 400,
                'body': json.dumps({
                    'error': 'Bad Request',
                    'message': 'Missing prompt in request'
                })
            }
        
        print(f"Processing prompt: {prompt}")
        print(f"Session ID: {session_id}")
        
        # AgentCore 用のペイロードを準備
        payload = json.dumps({"prompt": prompt})
        
        # AgentCore Runtime を呼び出し
        print("Invoking AgentCore Runtime...")
        response = bedrock_agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=runtime_arn,
            runtimeSessionId=session_id,
            payload=payload
        )
        
        print("Response received from AgentCore")
        
        # レスポンスを解析 - StreamingBody を処理
        agent_response = None
        
        if 'response' in response:
            response_body = response['response']
            
            # StreamingBody を処理
            if hasattr(response_body, 'read'):
                raw_data = response_body.read()
                if isinstance(raw_data, bytes):
                    agent_response = raw_data.decode('utf-8')
                else:
                    agent_response = str(raw_data)
            elif isinstance(response_body, list) and len(response_body) > 0:
                if isinstance(response_body[0], bytes):
                    agent_response = response_body[0].decode('utf-8')
                else:
                    agent_response = str(response_body[0])
            elif isinstance(response_body, bytes):
                agent_response = response_body.decode('utf-8')
            elif isinstance(response_body, str):
                agent_response = response_body
            else:
                agent_response = str(response_body)
        
        if not agent_response:
            agent_response = "No response from agent"
            print("Warning: No response extracted from AgentCore")
        
        print(f"Agent response received (length: {len(agent_response)} chars)")
        
        return {
            'statusCode': 200,
            'body': json.dumps({
                'response': agent_response,
                'sessionId': session_id
            }),
            'headers': {
                'Content-Type': 'application/json'
            }
        }
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"AWS ClientError: {error_code}")
        print(f"Error message: {error_message}")
        traceback.print_exc()
        
        return {
            'statusCode': 500,
            'body': json.dumps({
                'error': error_code,
                'message': error_message
            })
        }
    
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        print(f"Error type: {type(e).__name__}")
        traceback.print_exc()
        
        return {
            'statusCode': 500,
            'body': json.dumps({
                'error': 'InternalError',
                'message': str(e),
                'type': type(e).__name__
            })
        }

In [ ]:
# デプロイパッケージを作成
import zipfile
import os
from pathlib import Path

zip_filename = 'lambda_agentcore_invoker.zip'
lambda_file = 'lambda_agentcore_invoker.py'

print(f"Creating Lambda deployment package...")

# ファイルが存在することを確認
if not Path(lambda_file).exists():
    raise FileNotFoundError(f"{lambda_file} not found. Run the previous cell first.")

# 古い zip があれば削除
if Path(zip_filename).exists():
    Path(zip_filename).unlink()
    print(f"Removed old {zip_filename}")

# 新しい zip ファイルを作成
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(lambda_file, arcname=lambda_file)

print(f"Created Lambda deployment package: {zip_filename}")

# zip ファイルをバイトとして読み込み
with open(zip_filename, 'rb') as f:
    lambda_zip_content = f.read()

print(f"Package size: {len(lambda_zip_content):,} bytes")

In [ ]:
# ADOT Layer と X-Ray トレーシングを有効にして Lambda 関数を作成または更新
lambda_client = boto3.client('lambda', region_name=region)
lambda_function_name = f"agentcore-mcp-invoker-{agent_name}"

print(f"Lambda Configuration:")
print(f"  Function Name: {lambda_function_name}")
print(f"  Runtime ARN:   {agent_runtime_arn}")
print(f"  ADOT Layer:    {adot_layer_arn if adot_layer_arn else 'Not configured'}")
print(f"  AWS Region:    {region}")
print()

lambda_config = {
    'FunctionName': lambda_function_name,
    'Runtime': 'python3.12',
    'Role': lambda_role_arn,
    'Handler': 'lambda_agentcore_invoker.lambda_handler',
    'Code': {'ZipFile': lambda_zip_content},
    'Description': 'Lambda function to invoke AgentCore Runtime with MCP servers and ADOT instrumentation',
    'Timeout': 300,
    'MemorySize': 512,
    'Environment': {
        'Variables': {
            'RUNTIME_ARN': agent_runtime_arn,
            'AWS_LAMBDA_EXEC_WRAPPER': '/opt/otel-instrument'  # ADOT 自動インストゥルメンテーションを有効化
        }
    },
    'TracingConfig': {
        'Mode': 'Active'  # X-Ray トレーシングを有効化
    }
}

# ADOT Layer が利用可能な場合は追加
if adot_layer_arn:
    lambda_config['Layers'] = [adot_layer_arn]

try:
    lambda_response = lambda_client.create_function(**lambda_config)
    lambda_function_arn = lambda_response['FunctionArn']
    print(f"Created Lambda function: {lambda_function_name}")
    print(f"Function ARN: {lambda_function_arn}")
except lambda_client.exceptions.ResourceConflictException:
    print(f"Function exists, updating...")
    
    # コードを更新
    lambda_client.update_function_code(
        FunctionName=lambda_function_name,
        ZipFile=lambda_zip_content
    )
    print(f"  Code updated")
    
    # コード更新が処理されるまで少し待機
    time.sleep(2)
    
    # 設定を更新
    update_config = {
        'FunctionName': lambda_function_name,
        'Environment': lambda_config['Environment'],
        'TracingConfig': lambda_config['TracingConfig'],
        'Timeout': lambda_config['Timeout'],
        'MemorySize': lambda_config['MemorySize']
    }
    
    # Layer が利用可能な場合は追加
    if adot_layer_arn:
        update_config['Layers'] = [adot_layer_arn]
    
    lambda_client.update_function_configuration(**update_config)
    print(f"  Configuration updated")
    
    lambda_function_arn = lambda_client.get_function(
        FunctionName=lambda_function_name
    )['Configuration']['FunctionArn']
    print(f"Updated existing Lambda function: {lambda_function_name}")
except Exception as e:
    print(f"Lambda creation/update error: {e}")
    raise e

# 関数がアクティブになるまで待機
print("\nWaiting for Lambda function to be active...")
try:
    waiter = lambda_client.get_waiter('function_active_v2')
    waiter.wait(FunctionName=lambda_function_name)
    print("Lambda function is active and ready!")
except Exception as e:
    print(f"Waiter note: {e}")
    print("Continuing...")

print(f"\nX-Ray Active Tracing is ENABLED")
if adot_layer_arn:
    print(f"ADOT Layer is ENABLED - Trace context will propagate to AgentCore Runtime")
else:
    print(f"ADOT Layer is NOT enabled - Trace propagation may be limited")

## ステップ 8: Lambda 関数の呼び出しとテスト

Lambda 関数を呼び出して統合をテストしましょう。

In [ ]:
# テストペイロード
test_payloads = [
    {"prompt": "What is AWS Lambda? Answer in one sentence."},
    {"prompt": "Name 2 AWS compute services."},
    {"prompt": "What is Amazon S3 used for? Be brief."}
]

print("Invoking Lambda function with test payloads...\n")

for i, payload in enumerate(test_payloads, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}: {payload['prompt']}")
    print('='*80)
    
    try:
        response = lambda_client.invoke(
            FunctionName=lambda_function_name,
            InvocationType='RequestResponse',
            Payload=json.dumps(payload)
        )
        
        response_payload = json.loads(response['Payload'].read())
        
        if 'FunctionError' in response:
            print(f"\nLambda Function Error!")
            print(f"Error Type: {response_payload.get('errorType', 'Unknown')}")
            print(f"Error Message: {response_payload.get('errorMessage', 'Unknown')}")
            continue
        
        if response_payload['statusCode'] == 200:
            body = json.loads(response_payload['body'])
            print(f"\nSuccess!")
            print(f"Session ID: {body.get('sessionId', 'N/A')}")
            
            response_text = body.get('response', '')
            print(f"\nAgent Response:\n{response_text[:300]}...") if len(response_text) > 300 else print(f"\nAgent Response:\n{response_text}")
        else:
            print(f"\nError Response (Status: {response_payload['statusCode']})")
            body = json.loads(response_payload['body'])
            print(f"Error: {body.get('error', 'Unknown')}")
            print(f"Message: {body.get('message', 'Unknown')}")
    
    except Exception as e:
        print(f"\nUnexpected error: {e}")
        import traceback
        traceback.print_exc()
    
    time.sleep(2)

print("\n" + "="*80)
print("All test invocations completed!")
print("\nTraces are being processed and will be available in CloudWatch within 1-2 minutes.")

## ステップ 9: CloudWatch で可観測性データを表示

ADOT インストゥルメンテーションでトレースを生成したので、Lambda から AgentCore Runtime への完全なエンドツーエンドトレースを CloudWatch コンソールで表示できます。

### CloudWatch 可観測性ダッシュボード

#### 1. CloudWatch Gen AI Observability ダッシュボード
![image.png](./image/image1.png)
エージェントパフォーマンスを表示するためのプライマリダッシュボード：
- **機能**: エージェントビュー、セッションビュー、スパンタイムラインを含むトレース
- **メトリクス**: トークン使用量、期間、エラー率、ツール呼び出し
- **トレース継続性**: ADOT Layer により、Lambda → AgentCore Runtime の接続されたトレースが表示されます

![image2.png](./image/image2.png)

#### 2. CloudWatch Logs
生の実行ログ：
- **Lambda Logs**: 関数実行の詳細
- **AgentCore Logs**: エージェント処理ステップと MCP サーバー通信

### 直接コンソールリンク

In [ ]:
# CloudWatch コンソール URL を生成
base_url = f"https://{region}.console.aws.amazon.com/cloudwatch"

urls = {
    "Gen AI Observability Dashboard": f"{base_url}/home?region={region}#/gen-ai-observability/agent-core/agents",
    "Lambda Function Logs": f"{base_url}/home?region={region}#logsV2:log-groups/log-group/$252Faws$252Flambda$252F{lambda_function_name}",
    "AgentCore Runtime Logs": f"{base_url}/home?region={region}#logsV2:log-groups/log-group/$252Faws$252Fbedrock-agentcore$252Fruntimes$252F{agent_id}-DEFAULT"
}

print("CloudWatch Observability Links\n")
for name, url in urls.items():
    print(f"{name}:")
    print(f"  {url}\n")

print("\nKey Observability Features:")
print("\n1. Gen AI Observability Dashboard:")
print("   - セッション期間、カウント、会話フロー")
print("   - トークン使用量（入力/出力）とコスト")
print("   - MCP ツール呼び出しトレース")
print("   - エラー率とレイテンシメトリクス")
print("   - Lambda から AgentCore へのエンドツーエンドトレース（ADOT Layer 使用時）")
print("\n2. CloudWatch Logs:")
print("   - リクエスト/レスポンスデータを含む Lambda 実行ログ")
print("   - エージェント処理ステップと意思決定")
print("   - MCP サーバー通信ログ")
print("   - OpenTelemetry インストゥルメンテーションデータ")

print("\nCloudWatch コンソールでログを表示する方法:")
print("\n  Lambda Logs:")
print("   1. 上記の 'Lambda Function Logs' リンクをクリック")
print("   2. 最新のログストリームを選択")
print("   3. 以下を含む実行詳細を表示:")
print("      - START/END RequestId マーカー")
print("      - Processing prompt メッセージ")
print("      - Response received 確認")
print("      - REPORT での Duration とメモリ使用量")
print("\n  AgentCore Logs:")
print("   1. 上記の 'AgentCore Runtime Logs' リンクをクリック")
print("   2. 'runtime-logs' ログストリームを探す")
print("   3. 以下を含むエージェント実行を表示:")
print("      - MCP ツール呼び出し")
print("      - エージェント推論ステップ")
print("      - OpenTelemetry トレースデータ")

if adot_layer_arn:
    print("\nADOT Layer Enabled - Trace Context Propagation:")
    print("   - トレースは Lambda → AgentCore Runtime の接続を表示")
    print("   - CloudWatch X-Ray トレースマップで完全なリクエストフローを表示")
    print("   - セッション ID とトレース ID がサービス間で伝播")
else:
    print("\nADOT Layer Not Enabled:")
    print("   - Lambda と AgentCore 間のトレースが分断される可能性")
    print("   - 完全な可観測性のために ADOT Layer の追加を検討してください")

## まとめとベストプラクティス

### 達成したこと

- **エージェントデプロイ**: AWS Docs + CDK サーバーを持つ MCP エージェントを作成し、AgentCore Runtime にデプロイ

- **Lambda 統合**: ホストされたエージェントを呼び出す Lambda 関数を構築

- **ADOT インストゥルメンテーション**: トレース伝播のために AWS Lambda Layer for OpenTelemetry を追加

- **CloudWatch Observability**: 監視のための Gen AI Observability を有効化

- **エンドツーエンドテスト**: Lambda → AgentCore → MCP フローを示すトレースを生成

### 主要な可観測性コンポーネント

1. **ADOT Lambda Layer**: Lambda からダウンストリームサービスへのトレースコンテキスト伝播を有効化
2. **X-Ray Active Tracing**: リクエストパスの可視化を提供
3. **Gen AI Traces**: ツール呼び出しを含む完全なスパンタイムライン
4. **CloudWatch Logs**: タイムスタンプ付きの詳細な実行ログ
5. **Performance Metrics**: 期間、トークン使用量、エラー率

### ベストプラクティス

1. **常に ADOT Layer を使用**: サービス境界を越えたトレース継続性を確保
2. **セッション ID**: 会話を追跡するために一貫したセッション ID を使用
3. **メトリクス監視**: 期間、トークン使用量、エラー率を追跡
4. **アラーム設定**: しきい値のための CloudWatch アラームを作成
5. **ログ保持**: 適切な保持期間を設定
6. **定期レビュー**: トレースを分析してボトルネックを特定

### 次のステップ

1. **ダッシュボードを探索**: CloudWatch Gen AI Observability にアクセス
2. **トレースを分析**: 個々のトレースタイムラインを確認し、Lambda → AgentCore 接続を検証
3. **アラームを作成**: エラー率とレイテンシのアラートを設定
4. **最適化**: トレースデータを使用してパフォーマンスを改善
5. **スケール**: トラフィックの増加に応じて監視と調整

### リソースのクリーンアップ

In [ ]:
# # リソースを削除するにはコメントを解除

# # Lambda 関数を削除
# try:
#     lambda_client.delete_function(FunctionName=lambda_function_name)
#     print(f"Deleted Lambda function: {lambda_function_name}")
# except Exception as e:
#     print(f"Lambda deletion note: {e}")

# # IAM ポリシーをデタッチして削除
# for policy_arn in managed_policies + [agentcore_policy_arn]:
#     try:
#         iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn=policy_arn)
#     except Exception as e:
#         print(f"Policy detach note: {e}")

# # IAM ロールを削除
# try:
#     iam_client.delete_role(RoleName=lambda_role_name)
#     print(f"Deleted IAM role: {lambda_role_name}")
# except Exception as e:
#     print(f"Role deletion note: {e}")

# # カスタムポリシーを削除
# try:
#     iam_client.delete_policy(PolicyArn=agentcore_policy_arn)
#     print(f"Deleted custom policy: {agentcore_policy_name}")
# except Exception as e:
#     print(f"Policy deletion note: {e}")

# # AgentCore Runtime を削除
# try:
#     agentcore_runtime.delete()
#     print(f"Deleted AgentCore Runtime agent: {agent_name}")
# except Exception as e:
#     print(f"AgentCore deletion note: {e}")

print("To delete resources, uncomment the code above and run this cell.")

## おめでとうございます！

以下を成功裏に達成しました：
- MCP 対応エージェントを Amazon Bedrock AgentCore Runtime にデプロイ
- トレース伝播のために ADOT Layer を持つ Lambda 関数を作成
- CloudWatch Gen AI Observability を設定
- CloudWatch コンソールでエンドツーエンドトレースを生成して表示

これで完全な可観測性を備えた本番対応の AI エージェントを構築できます！

### 追加リソース

- [Amazon Bedrock AgentCore Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/what-is-bedrock-agentcore.html)
- [CloudWatch Gen AI Observability Guide](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GenAI-observability.html)
- [AWS Lambda Layer for OpenTelemetry](https://aws-otel.github.io/docs/getting-started/lambda)
- [AWS X-Ray Documentation](https://docs.aws.amazon.com/xray/latest/devguide/)
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/)